In [6]:
import csv

csvFile = open("BCHAIN-MKPRU.csv", "r")
reader = csv.reader(csvFile)
date={}
bitcoin = {}
i=0
for item in reader:
    if reader.line_num == 1:
        continue
    bitcoin[i] = float(item[1])
    date[i]=item[0]
    i=i+1

csvFile.close()

csvFile = open("LBMA-GOLD.csv", "r")
reader = csv.reader(csvFile)
j=0
gold = {}
for item in reader:
    if reader.line_num == 1:
        continue
    while(item[0]!=date[j]):
        gold[j] = 0
        j=j+1
    if(item[1]==''): gold[j]=0
    else: gold[j]=float(item[1])
    j=j+1

csvFile.close()

#print(gold)
#print(bitcoin)


In [7]:
def buy(C, G, B, date, kind, amount, allin=False):
    trade=0
    price=0
    a=0
    if(kind=='gold'):
        if(gold[date]==0): return 0
        a=ag
        price=gold[date]
        trade=amount*gold[date]*(1+a)
    if(kind=='bitcoin'):
        a=ab
        price=bitcoin[date]
        trade=amount*bitcoin[date]*(1+a)
    if(C<trade or allin==True):
        amount=C/(price*(1+a))
        C=0
        if(kind=='gold'): G=G+amount
        if(kind=='bitcoin'): B=B+amount
    else:
        C=C-trade
        if(kind=='gold'): G=G+amount
        if(kind=='bitcoin'): B=B+amount
    return C, G, B, amount

def sell(C, G, B, date, kind, amount, allin=False):
    price=0
    a=0
    if(kind=='gold'):
        if(gold[date]==0): return C, G, B, 0
        a=ag
        if(amount>G or allin==True): amount=G
        G=G-amount
        price=amount*gold[date]
    if(kind=='bitcoin'):
        a=ab
        if(amount>B or allin==True): amount=B
        B=B-amount
        price=amount*bitcoin[date]
    trade=price-price*a
    C=C+trade
    return C, G, B, amount

def average(start, kind, days, decay_rate=0.95):
    if(kind=='gold'):
        price=0
        count=0

        weight_sum=0
        
        now=start
        while(days>0):
            if(now<0):
                break
            if(gold[now]==0):
                days=days-1
                now=now-1
            else:
                # price=price+gold[now]
                
                factor=decay_rate**count
                price=price+gold[now]*factor
                weight_sum+=factor
                
                count=count+1
                now=now-1
                days=days-1
        if(count==0): return 0
        # return (price/count)
        return (price/weight_sum)
    if(kind=='bitcoin'):
        price=0
        count=0

        weight_sum=0

        now=start
        while(days>0):
            if(now<0):
                break
            # price=price+bitcoin[now]

            factor=decay_rate**count
            price=price+bitcoin[now]*factor
            weight_sum+=factor

            count=count+1
            now=now-1
            days=days-1
        if(count==0): return 0
        # return (price/count)
        return (price/weight_sum)
    
def rate(start, kind, P, L, decay_rate=0.95):
    priceP=average(start, kind, P, decay_rate)
    priceL=average(start, kind, L, decay_rate)
    if(priceL==0): return 0
    return (priceL-priceP)/priceP

In [20]:
def find_LP(C,B,G,i_th_day,old_L,old_P,mode):
    ag=0.01
    ab=0.02
    Lnum=range(50)
    Pnum=range(100)
    cf=0
    lf=0
    pf=0
    for l in Lnum:
        for p in Pnum:
            if(l>=p): continue
            c1,c2,c3=result(C=C, B=B, G=G, Days=i_th_day-1, ag=ag, ab=ab, L=l, P=p,
                        sellGold=0, buyGold=0, sellbitcoin=0, buybitcoin=0, decay_rate=0.95)
            if(c1>cf):
                pf=p
                lf=l
    # new L, P will average with old L,P
    if mode == 1:
        pf = round((pf+old_P)/2)
        lf = round((lf+old_L)/2)
    print(pf,lf)
    return lf, pf

In [21]:
def result(C, B, G, Days, ag, ab, L, P, sellGold=0, buyGold=0, sellbitcoin=0, buybitcoin=0, decay_rate=0.95):
    for i in range(Days):
        if(i<P): continue
        rateGold=rate(i, 'gold', P, L,decay_rate)
        rateBitcoin=rate(i, 'bitcoin', P, L,decay_rate)
        if(gold[i]!=0):
            if(rateGold>0 and rateBitcoin<=0):
                C, G, B, amount=sell(C, G, B, i, 'bitcoin', amount=0, allin=True)
                C, G, B, amount=buy(C, G, B, i, 'gold', amount=0, allin=True)
            if(rateGold<=0 and rateBitcoin>0):
                C, G, B, amount=sell(C, G, B, i, 'gold', amount=0, allin=True)
                C, G, B, amount=buy(C, G, B, i, 'bitcoin', amount=0, allin=True)
            if(rateGold>0 and rateBitcoin>0):
                total=rateGold+rateBitcoin
                aGold=rateGold/total
                aBitcoin=rateBitcoin/total
                Cgold=aGold*C
                Cbitcoin=aBitcoin*C
                C, G, B, amount=buy(Cgold, G, B, i, 'gold', amount=0, allin=True)
                C, G, B, amount=buy(Cbitcoin, G, B, i, 'bitcoin', amount=0, allin=True)
            if(rateGold<=0 and rateBitcoin<=0):
                C, G, B, amount=sell(C, G, B, i, 'bitcoin', amount=0, allin=True)
                C, G, B, amount=sell(C, G, B, i, 'gold', amount=0, allin=True)
        else:
            if(rateBitcoin>0):
                C, G, B, amount=buy(C, G, B, i, 'bitcoin', amount=0, allin=True)
            if(rateBitcoin<=0):
                C, G, B, amount=sell(C, G, B, i, 'bitcoin', amount=0, allin=True)
    C, G, B, amount=sell(C, G, B, Days-1, 'gold', amount=0, allin=True)
    C, G, B, amount=sell(C, G, B, Days-1, 'bitcoin', amount=0, allin=True)
    return C,G,B

In [22]:
def final_result(C, B, G, Days, ag, ab, sellGold=0, buyGold=0, sellbitcoin=0, buybitcoin=0, decay_rate=0.95):
    L,P = 5,10
    for i in range(Days):
        if(i<P and i<20): continue
        if(i%(150+P)==0):
            L,P = find_LP(C,B,G,i,L,P,mode=1)
        rateGold=rate(i, 'gold', P, L,decay_rate)
        rateBitcoin=rate(i, 'bitcoin', P, L,decay_rate)
        if(gold[i]!=0):
            if(rateGold>0 and rateBitcoin<=0):
                C, G, B, amount=sell(C, G, B, i, 'bitcoin', amount=0, allin=True)
                C, G, B, amount=buy(C, G, B, i, 'gold', amount=0, allin=True)
            if(rateGold<=0 and rateBitcoin>0):
                C, G, B, amount=sell(C, G, B, i, 'gold', amount=0, allin=True)
                C, G, B, amount=buy(C, G, B, i, 'bitcoin', amount=0, allin=True)
            if(rateGold>0 and rateBitcoin>0):
                total=rateGold+rateBitcoin
                aGold=rateGold/total
                aBitcoin=rateBitcoin/total
                Cgold=aGold*C
                Cbitcoin=aBitcoin*C
                C, G, B, amount=buy(Cgold, G, B, i, 'gold', amount=0, allin=True)
                C, G, B, amount=buy(Cbitcoin, G, B, i, 'bitcoin', amount=0, allin=True)
            if(rateGold<=0 and rateBitcoin<=0):
                C, G, B, amount=sell(C, G, B, i, 'bitcoin', amount=0, allin=True)
                C, G, B, amount=sell(C, G, B, i, 'gold', amount=0, allin=True)
        else:
            if(rateBitcoin>0):
                C, G, B, amount=buy(C, G, B, i, 'bitcoin', amount=0, allin=True)
            if(rateBitcoin<=0):
                C, G, B, amount=sell(C, G, B, i, 'bitcoin', amount=0, allin=True)
        if (i%100==0): print(i)
    C, G, B, amount=sell(C, G, B, Days-1, 'gold', amount=0, allin=True)
    C, G, B, amount=sell(C, G, B, Days-1, 'bitcoin', amount=0, allin=True)
    return C,G,B

In [ ]:
ag=0.01
ab=0.02
c1,c2,c3=final_result(C=1000, B=0, G=0, Days=len(bitcoin), ag=ag, ab=ab, sellGold=0, buyGold=0, sellbitcoin=0, buybitcoin=0)
c1

100
54 27
200
76 38
88 44
94 46
96 48
98 48
98 48
300
400
98 48
500
600
700
98 48
800
900
98 48
1000
1100
1200
74 48
1300
86 48
1400
68 48
1500
84 48
1600
92 48


In [17]:
ag=0.01
ab=0.02
c1,c2,c3=result(C=1000, B=0, G=0, Days=len(bitcoin), ag=ag, ab=ab, L=40, P=70,
                        sellGold=0, buyGold=0, sellbitcoin=0, buybitcoin=0)
c1

46009.43753240488